In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers

C:\Users\ADMIN\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\ADMIN\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
train_dir = r'C:\Users\ADMIN\Desktop\Coastline Detection\Coastline-Detection-Summer-Project-master\ML Based\urban-nonurban\data\train'
validation_dir = r'C:\Users\ADMIN\Desktop\Coastline Detection\Coastline-Detection-Summer-Project-master\ML Based\urban-nonurban\data\val'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling for validation data

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Size for VGGNet
    batch_size=20,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary')

Found 318 images belonging to 2 classes.
Found 78 images belonging to 2 classes.


c:\Users\ADMIN\anaconda3\lib\site-packages\keras\preprocessing\image.py:202: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn(


In [4]:
from tensorflow.keras.layers import Input

model = models.Sequential([
    Input(shape=(224, 224, 3)),  # Specify the input shape here
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # For binary classification
])

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),  # Correct parameter name
              metrics=['accuracy'])


In [5]:
history = model.fit(
    train_generator,  
    epochs=20,
    validation_data=validation_generator,
    )  

Epoch 1/20
16/16 [==============================] - 9s 320ms/step - loss: 0.6643 - accuracy: 0.6352 - val_loss: 0.5297 - val_accuracy: 0.7564
Epoch 2/20
16/16 [==============================] - 5s 296ms/step - loss: 0.6113 - accuracy: 0.6509 - val_loss: 0.4994 - val_accuracy: 0.7949
Epoch 3/20
16/16 [==============================] - 5s 300ms/step - loss: 0.5877 - accuracy: 0.7264 - val_loss: 0.5752 - val_accuracy: 0.6795
Epoch 4/20
16/16 [==============================] - 5s 305ms/step - loss: 0.5616 - accuracy: 0.7484 - val_loss: 0.6128 - val_accuracy: 0.6667
Epoch 5/20
16/16 [==============================] - 5s 306ms/step - loss: 0.5595 - accuracy: 0.7421 - val_loss: 0.4617 - val_accuracy: 0.7949
Epoch 6/20
16/16 [==============================] - 5s 303ms/step - loss: 0.5515 - accuracy: 0.7547 - val_loss: 0.4208 - val_accuracy: 0.8205
Epoch 7/20
16/16 [==============================] - 5s 297ms/step - loss: 0.5393 - accuracy: 0.7516 - val_loss: 0.4468 - val_accuracy: 0.7949
Epoch 

testing different models

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50, MobileNetV2, EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from sklearn.metrics import classification_report, accuracy_score

In [4]:
def build_model(model_name, input_shape=(224, 224, 3)):
    if model_name == 'CustomCNN':
        base_model = Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten()  # This correctly flattens the output for the Dense layer
        ])
    else:
        if model_name == 'ResNet50':
            pre_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
        elif model_name == 'MobileNetV2':
            pre_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
        elif model_name == 'EfficientNetB0':
            pre_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
        base_model = Sequential([
            pre_model,
            GlobalAveragePooling2D()  # Ensures compatibility with the GlobalAveragePooling2D layer
        ])
    
    model = Sequential([
        base_model,
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Now, let's initialize the models correctly
input_shape=(224, 224, 3)
models_to_train = {
    #'ResNet50': build_model('ResNet50', input_shape),
    'CustomCNN': build_model('CustomCNN', input_shape),
    #'MobileNetV2': build_model('MobileNetV2', input_shape),
    #'EfficientNetB0': build_model('EfficientNetB0', input_shape)
}


In [5]:
results = {}
for name, model in models_to_train.items():
    print(f"Training {name}...")
    # Assuming train_generator and validation_generator are defined as before
    history = model.fit(train_generator,
                        epochs=20,  # Adjust based on your needs
                        validation_data=validation_generator)
    
    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(validation_generator)
    results[name] = {'Validation Loss': val_loss, 'Validation Accuracy': val_accuracy}

# Print the comparison
for name, metrics in results.items():
    print(f"{name} - Loss: {metrics['Validation Loss']}, Accuracy: {metrics['Validation Accuracy']}")

Training CustomCNN...
Epoch 1/20
16/16 [==============================] - 8s 310ms/step - loss: 0.6881 - accuracy: 0.6321 - val_loss: 0.5095 - val_accuracy: 0.8462
Epoch 2/20
16/16 [==============================] - 5s 303ms/step - loss: 0.6067 - accuracy: 0.7138 - val_loss: 0.4941 - val_accuracy: 0.7692
Epoch 3/20
16/16 [==============================] - 5s 297ms/step - loss: 0.5666 - accuracy: 0.7327 - val_loss: 0.4377 - val_accuracy: 0.8333
Epoch 4/20
16/16 [==============================] - 5s 299ms/step - loss: 0.5439 - accuracy: 0.7547 - val_loss: 0.4133 - val_accuracy: 0.7821
Epoch 5/20
16/16 [==============================] - 5s 297ms/step - loss: 0.5231 - accuracy: 0.7547 - val_loss: 0.3935 - val_accuracy: 0.8205
Epoch 6/20
16/16 [==============================] - 5s 301ms/step - loss: 0.5433 - accuracy: 0.7421 - val_loss: 0.4211 - val_accuracy: 0.7949
Epoch 7/20
16/16 [==============================] - 5s 299ms/step - loss: 0.5303 - accuracy: 0.7704 - val_loss: 0.3808 - val_a

In [6]:
model.save('natural-built-classifier.h5')